In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
Phase_1 =  False
Phase_2 = False
Phase_3 = False
Phase_4 = False

DATA_FILE = "sc2021_train_deals.csv"
TARGET = "target"

# Range of values for selection in phase 1 :
import numpy as np
N_ESTIMATORS_RANG = list(range(190,250,10)) #list(range(100,1000,10))
RANG_LEARNING_RATE =  list(np.arange(0.02,0.04,0.001)) #list(np.arange(0.01,0.1,0.01)) 

RSCV_N_TRIALS = 100
N_TRIALS = 100
SEED = 82736

# Path definition
import pathlib
DATA_DIR = pathlib.Path(".")
LOG_PATH = pathlib.Path("./log/")
NAME_RUN_LOG = 'Run_CATBOOST_Logs.log'

AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]
CAT_COLS = ["material_code", "company_code",  "region",  "month"]

In [95]:
params = { 
    'n_estimators': 208,# 220, # Et1 - 208,# !!! 287 #
    'learning_rate': 0.05, # 0.03, # Et1 - 0.05, # !!! 0.021 #
    'depth': 4,         #6,
    'cat_features': CAT_COLS,
    'random_state' : SEED 
    }

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error, make_scorer

import numpy as np
# импортируем класс TimeSeriesSplit 
# реализующий стратегию перекрестной проверки на временных рядах
# и класс RandomizedSearchCV для поиска гиперпараметров
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from tqdm.notebook import tqdm
from time import time
import datetime
import warnings
import dill
import math
from sklearn.pipeline import Pipeline
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntUniformDistribution, UniformDistribution

warnings.simplefilter(action='ignore', category=FutureWarning)
# увеличиваем количество отображаемых столбцов
pd.set_option('display.max_columns', 100)

In [5]:
start_time = time()
print('Время старта скрипта: {}'.format(datetime.datetime.now().strftime("%d-%m-%Y %H:%M")))

Время старта скрипта: 24-12-2021 11:18


In [6]:
BASE_LOG = 4.3 # math.e

def exp_b_m1(x, base, r=4):
    return np.round(base**x - 1, r)

def log_b_1p(x, base):
    x.clip(lower=1.0e-200).apply(lambda y: math.log(y+1, base))
    return 

In [7]:
# загружаем набор
data = pd.read_csv(DATA_DIR.joinpath(DATA_FILE), parse_dates=["month", "date"])

In [8]:
group_ts = data.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)
#group_ts.head()

In [9]:
# 
def get_features_Cv(df: pd.DataFrame, month: pd.Timestamp, N=1, MDAD_1=2,  MNGR_GRP_MDAD=0) -> pd.DataFrame:
    """Calculate features for `month`."""

    start_period = month - pd.offsets.MonthBegin(N)
    end_period = month - pd.offsets.MonthBegin(1)

    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    # формируем лаги за N месяцев
    features[[f"vol_tm{i}" for i in range(N, 0, -1)]] = df.loc[:, start_period:end_period].copy()
 
    # Добавление ГРУППОВЫХ скользящих средних
    gr = "country"
    period = 10
    df2 = df.copy()
    df2[df2.columns.to_list()] = \
                           df2.groupby(level=gr).transform(lambda x: x.mean())
    grp_manager_roll_mean = df2.rolling(period, axis=1, min_periods=1)
    features = \
        features.join( grp_manager_roll_mean.mean().iloc[:, -1].rename(gr+"_grp_mean_"+str(period)))

    #  MEAN
    for period in [2,3,12]: #range(2,13,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.mean().iloc[:, -1].rename("avg_"+str(period)))

    #  median
    for period in [2,3,4,12]: # 10,11,12]: #range(2,13,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.median().iloc[:, -1].rename("median"+str(period)))
                 
    #  MAX
    for period in range(2,4,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.max().iloc[:, -1].rename("max_"+str(period)))

    features["month"] = month.month

    return features

In [10]:
#cv_traine_range = pd.date_range("2019-01-01", "2020-07-01", freq="MS")

cv_traine_range = pd.date_range("2018-07-01", "2020-07-01", freq="MS")
cv_traine_range, len(cv_traine_range)

(DatetimeIndex(['2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01',
                '2018-11-01', '2018-12-01', '2019-01-01', '2019-02-01',
                '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01',
                '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01',
                '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01',
                '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01',
                '2020-07-01'],
               dtype='datetime64[ns]', freq='MS'),
 25)

In [11]:
%%time

full_features = {}

tmp = group_ts.clip(lower=1.0e-200).copy()
for i in tmp.columns:
    tmp[i] = tmp[i].clip(lower=1.0e-200).apply(lambda x: math.log(x+1, BASE_LOG))

dataset_features = []
for target_month in tqdm(cv_traine_range):
    features = get_features_Cv(tmp, target_month)
    features[TARGET] = group_ts[target_month]
    dataset_features.append(features.reset_index())
full_features = pd.concat(dataset_features, ignore_index=True)

#CAT_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month"]
CAT_COLS = ["material_code", "company_code",  "region",  "month"]
# создаем список  переменных
FTS_COLS = full_features.columns.tolist()
FTS_COLS.remove(TARGET)
#################################################################################################################################
print(FTS_COLS)

  0%|          | 0/25 [00:00<?, ?it/s]

['material_code', 'company_code', 'country', 'region', 'manager_code', 'month', 'vol_tm1', 'country_grp_mean_10', 'avg_2', 'avg_3', 'avg_12', 'median2', 'median3', 'median4', 'median12', 'max_2', 'max_3']
Wall time: 22.9 s


In [12]:
FTS_COLS.remove("country")
FTS_COLS.remove("manager_code")
print(FTS_COLS)

['material_code', 'company_code', 'region', 'month', 'vol_tm1', 'country_grp_mean_10', 'avg_2', 'avg_3', 'avg_12', 'median2', 'median3', 'median4', 'median12', 'max_2', 'max_3']


In [13]:
cv_features = full_features.copy()

In [14]:
# 18 групп - блоки по 1 наблюдению(в 1 наблюдении данные 941 временного ряда)
N_timeseries = 941 # в 1 наблюдении данные 941 временного ряда
start_train = 1  # 

test_size=N_timeseries #*3
#max_train_size = N_timeseries*6
n_splits= 17 #int(len(full_features2)/(test_size))-1 - start_train # 

tscv = TimeSeriesSplit(n_splits=n_splits, 
                       #max_train_size=max_train_size, 
                       test_size=test_size)

In [15]:
# 6 групп
N_timeseries = 941
start_train = 1  # 

test_size=N_timeseries *3
#max_train_size = N_timeseries*6
n_splits= 5 #int(len(full_features2)/(test_size))-1 - start_train # 

tscv = TimeSeriesSplit(n_splits=n_splits, 
                       #max_train_size=max_train_size, 
                       test_size=test_size)

In [16]:
for train_index, test_index in tscv.split(cv_features): # tscv.split(X): # tscv.split(cv_features)
    print("TRAIN: ", train_index[0],"...",train_index[-1])
    print("TEST: ", test_index[0],"...",test_index[-1])

    X_train, X_test = cv_features.iloc[train_index], cv_features.iloc[test_index]
    y_train, y_test = cv_features.iloc[train_index][TARGET], cv_features.iloc[test_index][TARGET]
    
    print('Обучающий набор: %f' % (len(X_train)/941)) #test_size))
    print('Тестовый набор: %d' % (len(X_test)/941)) #test_size))

    #display(X_train)
    display(X_test)
print("tscv.get_n_splits()=",tscv.get_n_splits())

TRAIN:  0 ... 9409
TEST:  9410 ... 12232
Обучающий набор: 10.000000
Тестовый набор: 3


,material_code,company_code,country,region,manager_code,month,vol_tm1,country_grp_mean_10,avg_2,avg_3,avg_12,median2,median3,median4,median12,max_2,max_3,target
9410,124,7278,Россия,Респ. Татарстан,17460,5,3.582677,1.819675,3.609263e+00,3.618125,3.715942,3.609263,3.635850,3.635850,3.700882,3.635850,3.635850,103.0
9411,133,0,Белоруссия,Минская обл.,10942,5,2.475580,2.001837,1.237790e+00,0.825193,0.441160,1.237790,0.000000,0.000000,0.000000,2.475580,2.475580,98.0
9412,133,0,Белоруссия,Могилевская обл.,10942,5,3.184110,2.001837,3.293273e+00,3.199767,1.828690,3.293273,3.184110,3.098432,2.550012,3.402436,3.402436,145.0
9413,133,0,Белоруссия,г. Минск,10942,5,0.000000,2.001837,4.440892e-16,0.000000,2.299977,0.000000,0.000000,0.000000,2.682149,0.000000,0.000000,0.0
9414,133,0,Казахстан,г. Нур-Султан,13301,5,2.545958,2.050522,1.272979e+00,1.747182,1.655577,1.272979,2.545958,2.545958,2.450118,2.545958,2.695588,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12228,986,9943,Россия,Смоленская обл.,17460,7,0.000000,1.870797,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
12229,998,0,Россия,Ленинградская обл.,18079,7,1.643954,1.870797,1.436176e+00,1.459576,0.866310,1.436176,1.506377,1.367388,1.228398,1.643954,1.643954,5.0
12230,998,3380,Россия,Ленинградская обл.,14956,7,3.293550,1.870797,3.310007e+00,3.247356,1.062902,3.310007,3.293550,3.207803,0.000000,3.326464,3.326464,121.0
12231,998,5410,Россия,г. Санкт-Петербург,14956,7,3.164043,1.870797,3.278411e+00,3.281577,3.189806,3.278411,3.287907,3.340344,3.164043,3.392780,3.392780,120.0


TRAIN:  0 ... 12232
TEST:  12233 ... 15055
Обучающий набор: 13.000000
Тестовый набор: 3


,material_code,company_code,country,region,manager_code,month,vol_tm1,country_grp_mean_10,avg_2,avg_3,avg_12,median2,median3,median4,median12,max_2,max_3,target
12233,124,7278,Россия,Респ. Татарстан,17460,8,0.000000,1.909321,1.420229e+00,2.008189,3.292768,1.420229,2.840458,3.012284,3.635850,2.840458,3.184110,0.0
12234,133,0,Белоруссия,Минская обл.,10942,8,2.840458,2.032495,2.934967e+00,3.006755,0.957987,2.934967,3.029477,2.934967,0.000000,3.029477,3.150331,145.0
12235,133,0,Белоруссия,Могилевская обл.,10942,8,0.000000,2.032495,0.000000e+00,1.138890,1.830681,0.000000,0.000000,1.592055,2.550012,0.000000,3.416670,0.0
12236,133,0,Белоруссия,г. Минск,10942,8,0.000000,2.032495,4.440892e-16,0.000000,1.500504,0.000000,0.000000,0.000000,1.043635,0.000000,0.000000,41.0
12237,133,0,Казахстан,г. Нур-Султан,13301,8,2.545958,1.979334,2.545958e+00,1.697305,1.867740,2.545958,2.545958,2.545958,2.545958,2.545958,2.545958,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15051,986,9943,Россия,Смоленская обл.,17460,10,0.000000,1.983002,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
15052,998,0,Россия,Ленинградская обл.,18079,10,1.643954,1.983002,1.436176e+00,1.366916,1.013723,1.436176,1.228398,1.436176,1.228398,1.643954,1.643954,10.0
15053,998,3380,Россия,Ленинградская обл.,14956,10,3.270695,1.983002,3.303894e+00,3.300446,1.888013,3.303894,3.293550,3.293550,3.067405,3.337093,3.337093,115.0
15054,998,5410,Россия,г. Санкт-Петербург,14956,10,3.012755,1.983002,3.150331e+00,3.196190,3.223058,3.150331,3.287907,3.225975,3.285062,3.287907,3.287907,120.0


TRAIN:  0 ... 15055
TEST:  15056 ... 17878
Обучающий набор: 16.000000
Тестовый набор: 3


...

TRAIN:  0 ... 17878
TEST:  17879 ... 20701
Обучающий набор: 19.000000
Тестовый набор: 3


,material_code,company_code,country,region,manager_code,month,vol_tm1,country_grp_mean_10,avg_2,avg_3,avg_12,median2,median3,median4,median12,max_2,max_3,target
17879,124,7278,Россия,Респ. Татарстан,17460,2,0.000000,2.039984,0.000000e+00,0.000000,1.406579,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
17880,133,0,Белоруссия,Минская обл.,10942,2,1.981586,2.319127,2.820453e+00,3.101171,2.591167,2.820453,3.659319,3.613546,3.089904,3.659319,3.662607,72.0
17881,133,0,Белоруссия,Могилевская обл.,10942,2,0.000000,2.319127,0.000000e+00,0.000000,1.084664,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
17882,133,0,Белоруссия,г. Минск,10942,2,0.000000,2.319127,4.440892e-16,0.859537,0.934021,0.000000,0.000000,1.289305,0.000000,0.000000,2.578611,0.0
17883,133,0,Казахстан,г. Нур-Султан,13301,2,0.000000,2.180109,1.312424e+00,1.773478,1.735392,1.312424,2.624847,2.624847,2.545958,2.624847,2.695588,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20697,986,9943,Россия,Смоленская обл.,17460,4,2.851255,2.034251,2.485209e+00,1.656806,0.414201,2.485209,2.119163,1.059581,0.000000,2.851255,2.851255,125.0
20698,998,0,Россия,Ленинградская обл.,18079,4,1.228398,2.034251,1.228398e+00,1.228398,1.261892,1.228398,1.228398,1.228398,1.228398,1.228398,1.228398,0.0
20699,998,3380,Россия,Ленинградская обл.,14956,4,3.299146,2.034251,3.129569e+00,3.069976,3.135133,3.129569,2.959992,2.955390,3.264835,3.299146,3.299146,100.0
20700,998,5410,Россия,г. Санкт-Петербург,14956,4,3.563995,2.034251,3.563995e+00,3.430678,3.297505,3.563995,3.563995,3.364019,3.287907,3.563995,3.563995,100.0


TRAIN:  0 ... 20701
TEST:  20702 ... 23524
Обучающий набор: 22.000000
Тестовый набор: 3


,material_code,company_code,country,region,manager_code,month,vol_tm1,country_grp_mean_10,avg_2,avg_3,avg_12,median2,median3,median4,median12,max_2,max_3,target
20702,124,7278,Россия,Респ. Татарстан,17460,5,0.000000,2.026716,0.000000,0.000000,0.502047,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
20703,133,0,Белоруссия,Минская обл.,10942,5,4.099016,2.306797,3.943583,3.609542,3.287254,3.943583,3.788150,3.364806,3.363437,4.099016,4.099016,288.0
20704,133,0,Белоруссия,Могилевская обл.,10942,5,3.508804,2.306797,1.754402,1.169601,0.577123,1.754402,0.000000,0.000000,0.000000,3.508804,3.508804,62.0
20705,133,0,Белоруссия,г. Минск,10942,5,2.119163,2.306797,1.059581,0.706388,1.110618,1.059581,0.000000,0.000000,0.000000,2.119163,2.119163,0.0
20706,133,0,Казахстан,г. Нур-Султан,13301,5,0.000000,1.987240,1.272979,1.747182,1.735392,1.272979,2.545958,1.272979,2.545958,2.545958,2.695588,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23520,986,9943,Россия,Смоленская обл.,17460,7,3.045801,1.965926,3.045801,3.135756,1.198141,3.045801,3.045801,3.045801,0.000000,3.045801,3.315667,83.0
23521,998,0,Россия,Ленинградская обл.,18079,7,0.950418,1.965926,0.950418,0.633612,1.055401,0.950418,0.950418,0.950418,1.228398,0.950418,0.950418,9.0
23522,998,3380,Россия,Ленинградская обл.,14956,7,2.354279,1.965926,2.127558,2.473053,2.941557,2.127558,2.354279,2.759161,3.170765,2.354279,3.164043,50.0
23523,998,5410,Россия,г. Санкт-Петербург,14956,7,2.545958,1.965926,2.969369,3.034260,3.235676,2.969369,3.164043,3.278411,3.287907,3.392780,3.392780,0.0


tscv.get_n_splits()= 5


In [17]:
# создаем списки категориальных признаков
#cat_columns = data.select_dtypes(include='object').columns
cat_columns = CAT_COLS

# создаем список количественных признаков
#num_columns = train.select_dtypes(exclude='object').columns.tolist()
#num_columns =  [x for x in cv_features.columns.to_list() if x not in CAT_COLS + [TARGET]]
num_columns =  [x for x in FTS_COLS if x not in CAT_COLS + [TARGET]]
print(cat_columns)
print(num_columns)

['material_code', 'company_code', 'region', 'month']
['vol_tm1', 'country_grp_mean_10', 'avg_2', 'avg_3', 'avg_12', 'median2', 'median3', 'median4', 'median12', 'max_2', 'max_3']


> # ********  Phase 1 *************

In [18]:
# обучаем модель, используя логарифмирование зависимой 

train = cv_features[FTS_COLS]
y_train = cv_features[TARGET].clip(lower=1.0e-200).apply(lambda x: math.log(x+1, BASE_LOG))

In [19]:
if Phase_1:
    params2 = params   
    # задаем конвейер
    ml_pipe = Pipeline([('ml', CatBoostRegressor(**params) )])

In [20]:
if Phase_1:
    # задаем сетку гиперпараметров
    param_grid = [{'ml__learning_rate': RANG_LEARNING_RATE,
                   'ml__n_estimators' : N_ESTIMATORS_RANG}]

In [21]:
rmsle = lambda y_true, y_pred: \
            np.sqrt(mean_squared_log_error(
                        np.clip(y_true,0,None), 
                        np.clip(y_pred,0,None)
                    ))

In [22]:
%%time

if Phase_1:
    # создаем экземпляр класса RandomizedSearchCV или GridSearchCV, передав конвейер,
    # сетку гиперпараметров и указав количество
    # блоков перекрестной проверки, отключив запись метрик 
    # для обучающих блоков перекрестной проверки в атрибут cv_results_
    #gs = GridSearchCV(
    gs = RandomizedSearchCV( 
                      ml_pipe, 
                      param_grid, 
                      cv=tscv,  
                      scoring=make_scorer(rmsle, greater_is_better=False),
                      n_jobs=-1, 
                      return_train_score=False,
                      verbose=1,
                      random_state=SEED,
                    n_iter = RSCV_N_TRIALS
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
                     )
    # выполняем решетчатый поиск
    gs.fit(train, y_train)
    
    
    # записываем оптимальные значения гиперпараметров
    OPT_PARAMS_PIPE = gs.best_params_ 
    display(gs.best_params_, gs.best_score_)

Wall time: 0 ns


In [23]:
if Phase_1:
    #y_probas = gs.predict_proba(test)
    # смотрим наилучшие значения гиперпараметров
    print('Наилучшие значения гиперпараметров: {}'.format(OPT_PARAMS_PIPE))
    # смотрим наилучшее значение SCORE
    print('Наилучшее значение SCORE: {:.3f}'.format(gs.best_score_))
    # смотрим значение SCORE на тестовой выборке
    #print('Значение SCORE на тестовой выборке: {:.3f}'.format(
    #   roc_auc_score(y_test, y_probas[:, 1] )))

In [24]:
if Phase_1:
    # увеличиваем количество выводимых строк
    pd.set_option('display.max_rows', 300)

    # выводим результаты
    cv_results = pd.DataFrame(gs.cv_results_)[['mean_test_score',
                                               'param_ml__n_estimators',
                                               'param_ml__learning_rate',
                                               ]]
    cv_results = cv_results.sort_values(by='mean_test_score', ascending=False).reset_index(drop=True)
    for col in cv_results.columns[cv_results.columns.str.contains('enc')]:
        if col in 'mean_test_score':
            continue
        cv_results[col] = cv_results[col].apply(lambda x: type(x).__name__)
    display(cv_results)

> # ********  Phase 2,3,4 *************

In [25]:
if Phase_2 or Phase_3 or Phase_4:
    # задаем итоговый конвейер ПОКА ЧТО С НАЧАЛЬНЫМИ НАСТРОЙКАМИ
    ml_pipe2 = Pipeline([('ml', CatBoostRegressor(**params) )])

    # УСТАНОВКА В НОВЫЙ PIPE НАИЛУЧШИХ ЗНАЧЕНИЙ НАЙДЕННЫХ В ПРЕДЫДУЩЕЙ ФАЗЕ:  
    if Phase_1:  
        # присваиваем итоговому конвейеру оптимальные значения гиперпараметров
        print('Before modification ml__n_estimators ml_pipe2: ', ml_pipe2.get_params()['ml__n_estimators']) # 'n_estimators'
        print('Before modification ml__learning_rate ml_pipe2: ', ml_pipe2.get_params()['ml__learning_rate']) # 'n_estimators'
        ml_pipe2.set_params(**OPT_PARAMS_PIPE)
        print('After modification ml__n_estimators ml_pipe2: ', ml_pipe2.get_params()['ml__n_estimators'])
        print('After modification ml__learning_rate ml_pipe2: ', ml_pipe2.get_params()['ml__learning_rate'])

In [26]:
params2 = params3 = params4 = {}

if Phase_2: 
    params2 = {
        # (1) гиперпараметры, задающие структуру дерева:
       'ml__depth': IntUniformDistribution(1, 32),
       'ml__num_leaves': IntUniformDistribution(2, 1024),
       'ml__max_bin': IntUniformDistribution(1, 1024),  
       'ml__min_child_samples': IntUniformDistribution(1, 300),
       'ml__min_child_weight': UniformDistribution(0.01, 300)
        }

In [27]:
 # (2) гиперпараметры, привносящие случайность в ходе построения дерева:
if Phase_3: 
    params3 = {
               'ml__bagging_fraction': UniformDistribution(0.0, 1.0),# синонимы:subsample,...
               'ml__colsample_bytree': UniformDistribution(0.0, 1.0), # (0.3, 0.9), #синонимы:colsample_bytree,feature_fraction,...
               'ml__subsample': IntUniformDistribution(1, 81, 5), # 'синонимы:subsample',bagging_freq,...
             }

In [28]:
# (3) гиперпараметры, задающие регуляризацию функции потерь:
if Phase_4: 
    params4 = {
                'ml__reg_alpha': UniformDistribution(1e-3, 25.0), #0,01 или 0,1
                'ml__reg_lambda': UniformDistribution(1e-3, 25.0), #0,01 или 0,1
             }

In [29]:
if Phase_2 or Phase_3 or Phase_4:  
    # задаем новое пространство поиска
    param_distributions = {**params2,**params3,**params4}
    print(param_distributions)

In [30]:
if Phase_2 or Phase_3 or Phase_4:  
    # создаем экземпляр класса OptunaSearchCV
    optuna_search = OptunaSearchCV(
        ml_pipe2,
        param_distributions,
        scoring=make_scorer(rmsle, greater_is_better=False),
        n_trials=N_TRIALS,
        n_jobs=-1,
        refit=True,
        verbose=1,
        cv=tscv,   # CV_O,
        random_state=SEED)

In [31]:
%%time
if Phase_2 or Phase_3 or Phase_4:  
    # выполняем оптимизацию
    optuna_search.fit(train, y_train)

Wall time: 0 ns


In [32]:
if Phase_2 or Phase_3 or Phase_4:  
    # записываем оптимальные значения гиперпараметров optuna
    opt_optuna_params_pipe = optuna_search.best_params_
    if Phase_1:
        opt_optuna_params_pipe['ml__learning_rate'] = OPT_PARAMS_PIPE['ml__learning_rate']
        opt_optuna_params_pipe['ml__n_estimators'] = OPT_PARAMS_PIPE['ml__n_estimators']

    # печатаем наилучшие значения гиперпараметров
    print('Наилучшие значения гиперпараметров: {}'.format(opt_optuna_params_pipe))
    # печатаем наилучшее значение SCORE
    print('Наилучшее значение SCORE: {:.3f}'.format(optuna_search.best_score_))
    # смотрим значение SCORE на тестовой выборке
    #print('SCORE на тестовой выборке: {:.3f}'.format(
    #    rsmle(y_test, optuna_search.predict_proba(test)[:, 1])))
    print('')
    with open(NAME_RUN_LOG, 'at', encoding='utf-8') as logfile:
        print('/nВременная метка окончания ФАЗЫ 2 : {}'.format(datetime.now().strftime("%d-%m-%Y %H:%M")), 
          file=logfile)
        print('Наилучшие значения гиперпараметров найденых optuna: \n{}'.format(opt_optuna_params_pipe), file=logfile)
        print('Наилучшее значение SCORE: {:.3f}'.format(optuna_search.best_score_), file=logfile)

#### Найденные на каждой фазе значения записываем в params = {...} в ячейке вверху ноутбука

> # ********  Final phase *************

In [33]:
# формируем полный массив меток и массив признаков

In [34]:
fulldata, y_fulldata = full_features[FTS_COLS]  , full_features[TARGET]

In [148]:
from sklearn.base import BaseEstimator
import random


class StochasticMultiModel(BaseEstimator):
    
    def __init__(self, clf, params, N=100,frac=0.5, rang_iter=0, seed=42):
        self.clf = clf
        self.params = params
        self.N = N
        self.seed = seed
        self.frac = frac
        self.rang_iter = rang_iter
        self.models = []
        
    def fit(self, x_train, y_train):
        for i in tqdm(range( self.N )):
            
            # выполняем случайный отбор наблюдений
            np.random.seed(i*self.seed)
            idx = np.random.choice(
                x_train.shape[0], 
                int(self.frac * len(x_train)), replace=False)
            new_x_train = np.take(x_train, idx, axis=0)
            new_y_train = np.take(y_train, idx, axis=0)
            
            random.randint(1,27)
            self.params['n_estimators'] = \
                self.params['n_estimators'] \
                + random.randint(0,self.rang_iter) \
                - random.randint(0,self.rang_iter)
            
            self.params['random_state'] = i*self.seed
            model = self.clf(**self.params)
            model.fit(new_x_train, new_y_train)
            self.models.append(model)
        return self
    
    def predict(self, test):
        preds_df = pd.DataFrame()
        for i in range( self.N ):
            preds_df['pred_' + str(i)] = \
            self.models[i].predict(test)
        return preds_df.mean(1)

In [166]:
params = { 
    'n_estimators': 208,# 220, # Et1 - 208,# !!! 287 #
    'learning_rate': 0.05, # 0.03, # Et1 - 0.05, # !!! 0.021 #
    'depth': 4,         #6,
    'cat_features': CAT_COLS,
    'random_state' : SEED 
    }
params['n_estimators'] = 217

final_ml_pipe= \
StochasticMultiModel(CatBoostRegressor, params, N=100,frac=0.5, rang_iter=0)
#CatBoostRegressor(**params)

In [167]:
%%time

# обучаем обновленную модель на всей обучающей выборке
final_ml_pipe.fit(
                fulldata[FTS_COLS], 
                y_fulldata.clip(lower=1.0e-200).apply(lambda x: math.log(x+1, BASE_LOG ))
                )

  0%|          | 0/100 [00:00<?, ?it/s]

0:	learn: 1.5099207	total: 27.6ms	remaining: 5.97s
1:	learn: 1.4770147	total: 57.8ms	remaining: 6.21s
2:	learn: 1.4454948	total: 84.3ms	remaining: 6.01s
3:	learn: 1.4166971	total: 110ms	remaining: 5.84s
4:	learn: 1.3902307	total: 135ms	remaining: 5.71s
5:	learn: 1.3651862	total: 156ms	remaining: 5.49s
6:	learn: 1.3419188	total: 179ms	remaining: 5.37s
7:	learn: 1.3209346	total: 203ms	remaining: 5.31s
8:	learn: 1.3011208	total: 229ms	remaining: 5.3s
9:	learn: 1.2829182	total: 262ms	remaining: 5.43s
10:	learn: 1.2664676	total: 314ms	remaining: 5.87s
11:	learn: 1.2510227	total: 348ms	remaining: 5.95s
12:	learn: 1.2363831	total: 379ms	remaining: 5.94s
13:	learn: 1.2230240	total: 408ms	remaining: 5.92s
14:	learn: 1.2107931	total: 438ms	remaining: 5.9s
15:	learn: 1.1992824	total: 469ms	remaining: 5.89s
16:	learn: 1.1890035	total: 505ms	remaining: 5.94s
17:	learn: 1.1793076	total: 540ms	remaining: 5.97s
18:	learn: 1.1701825	total: 574ms	remaining: 5.98s
19:	learn: 1.1622905	total: 607ms	remain

168:	learn: 1.0457623	total: 4.33s	remaining: 1.23s
169:	learn: 1.0456466	total: 4.37s	remaining: 1.21s
170:	learn: 1.0456117	total: 4.39s	remaining: 1.18s
171:	learn: 1.0454563	total: 4.41s	remaining: 1.15s
172:	learn: 1.0454012	total: 4.43s	remaining: 1.13s
173:	learn: 1.0452866	total: 4.46s	remaining: 1.1s
174:	learn: 1.0452398	total: 4.48s	remaining: 1.07s
175:	learn: 1.0451471	total: 4.5s	remaining: 1.05s
176:	learn: 1.0451111	total: 4.54s	remaining: 1.03s
177:	learn: 1.0450774	total: 4.58s	remaining: 1s
178:	learn: 1.0450297	total: 4.62s	remaining: 981ms
179:	learn: 1.0449079	total: 4.64s	remaining: 954ms
180:	learn: 1.0447401	total: 4.67s	remaining: 928ms
181:	learn: 1.0446220	total: 4.69s	remaining: 902ms
182:	learn: 1.0445612	total: 4.71s	remaining: 875ms
183:	learn: 1.0443531	total: 4.74s	remaining: 850ms
184:	learn: 1.0443186	total: 4.77s	remaining: 825ms
185:	learn: 1.0442485	total: 4.79s	remaining: 799ms
186:	learn: 1.0441263	total: 4.82s	remaining: 773ms
187:	learn: 1.043

113:	learn: 1.0637159	total: 3.1s	remaining: 2.8s
114:	learn: 1.0635852	total: 3.13s	remaining: 2.77s
115:	learn: 1.0633382	total: 3.15s	remaining: 2.75s
116:	learn: 1.0632670	total: 3.18s	remaining: 2.72s
117:	learn: 1.0631009	total: 3.21s	remaining: 2.7s
118:	learn: 1.0628299	total: 3.24s	remaining: 2.67s
119:	learn: 1.0627803	total: 3.28s	remaining: 2.65s
120:	learn: 1.0626607	total: 3.31s	remaining: 2.63s
121:	learn: 1.0624530	total: 3.34s	remaining: 2.6s
122:	learn: 1.0623776	total: 3.37s	remaining: 2.58s
123:	learn: 1.0623091	total: 3.4s	remaining: 2.55s
124:	learn: 1.0621404	total: 3.43s	remaining: 2.52s
125:	learn: 1.0619757	total: 3.46s	remaining: 2.5s
126:	learn: 1.0618626	total: 3.49s	remaining: 2.47s
127:	learn: 1.0618060	total: 3.51s	remaining: 2.44s
128:	learn: 1.0616313	total: 3.54s	remaining: 2.42s
129:	learn: 1.0615755	total: 3.57s	remaining: 2.39s
130:	learn: 1.0612906	total: 3.6s	remaining: 2.36s
131:	learn: 1.0611681	total: 3.63s	remaining: 2.33s
132:	learn: 1.06112

62:	learn: 1.0627009	total: 1.83s	remaining: 4.47s
63:	learn: 1.0620993	total: 1.86s	remaining: 4.45s
64:	learn: 1.0617369	total: 1.89s	remaining: 4.41s
65:	learn: 1.0613230	total: 1.92s	remaining: 4.39s
66:	learn: 1.0608634	total: 1.94s	remaining: 4.35s
67:	learn: 1.0603610	total: 1.97s	remaining: 4.31s
68:	learn: 1.0599723	total: 2s	remaining: 4.28s
69:	learn: 1.0595549	total: 2.02s	remaining: 4.24s
70:	learn: 1.0591884	total: 2.07s	remaining: 4.26s
71:	learn: 1.0589852	total: 2.1s	remaining: 4.23s
72:	learn: 1.0587049	total: 2.13s	remaining: 4.19s
73:	learn: 1.0584108	total: 2.15s	remaining: 4.16s
74:	learn: 1.0581186	total: 2.18s	remaining: 4.12s
75:	learn: 1.0579166	total: 2.2s	remaining: 4.08s
76:	learn: 1.0576913	total: 2.23s	remaining: 4.05s
77:	learn: 1.0573450	total: 2.25s	remaining: 4.01s
78:	learn: 1.0572000	total: 2.28s	remaining: 3.99s
79:	learn: 1.0569401	total: 2.31s	remaining: 3.96s
80:	learn: 1.0567675	total: 2.33s	remaining: 3.92s
81:	learn: 1.0564407	total: 2.36s	re

...

18:	learn: 1.1622033	total: 540ms	remaining: 5.63s
19:	learn: 1.1540518	total: 567ms	remaining: 5.58s
20:	learn: 1.1466017	total: 594ms	remaining: 5.54s
21:	learn: 1.1400717	total: 622ms	remaining: 5.51s
22:	learn: 1.1336287	total: 651ms	remaining: 5.49s
23:	learn: 1.1277014	total: 682ms	remaining: 5.48s
24:	learn: 1.1224242	total: 713ms	remaining: 5.48s
25:	learn: 1.1176538	total: 744ms	remaining: 5.47s
26:	learn: 1.1131555	total: 775ms	remaining: 5.45s
27:	learn: 1.1088513	total: 806ms	remaining: 5.44s
28:	learn: 1.1053245	total: 835ms	remaining: 5.41s
29:	learn: 1.1018138	total: 865ms	remaining: 5.39s
30:	learn: 1.0987957	total: 901ms	remaining: 5.41s
31:	learn: 1.0956882	total: 939ms	remaining: 5.43s
32:	learn: 1.0927862	total: 971ms	remaining: 5.41s
33:	learn: 1.0901652	total: 1s	remaining: 5.38s
34:	learn: 1.0877092	total: 1.03s	remaining: 5.34s
35:	learn: 1.0854069	total: 1.06s	remaining: 5.31s
36:	learn: 1.0834983	total: 1.08s	remaining: 5.28s
37:	learn: 1.0814818	total: 1.11s	

182:	learn: 1.0367816	total: 5.23s	remaining: 972ms
183:	learn: 1.0366695	total: 5.26s	remaining: 943ms
184:	learn: 1.0365712	total: 5.29s	remaining: 914ms
185:	learn: 1.0364361	total: 5.31s	remaining: 885ms
186:	learn: 1.0362826	total: 5.34s	remaining: 856ms
187:	learn: 1.0361821	total: 5.36s	remaining: 827ms
188:	learn: 1.0361174	total: 5.39s	remaining: 798ms
189:	learn: 1.0359207	total: 5.42s	remaining: 771ms
190:	learn: 1.0358103	total: 5.45s	remaining: 742ms
191:	learn: 1.0357679	total: 5.48s	remaining: 713ms
192:	learn: 1.0357247	total: 5.5s	remaining: 685ms
193:	learn: 1.0356932	total: 5.53s	remaining: 656ms
194:	learn: 1.0355427	total: 5.56s	remaining: 627ms
195:	learn: 1.0354583	total: 5.58s	remaining: 598ms
196:	learn: 1.0353251	total: 5.62s	remaining: 570ms
197:	learn: 1.0352383	total: 5.64s	remaining: 541ms
198:	learn: 1.0351736	total: 5.67s	remaining: 513ms
199:	learn: 1.0349346	total: 5.69s	remaining: 484ms
200:	learn: 1.0347574	total: 5.72s	remaining: 455ms
201:	learn: 1

134:	learn: 1.0444588	total: 3.69s	remaining: 2.24s
135:	learn: 1.0443801	total: 3.72s	remaining: 2.21s
136:	learn: 1.0441924	total: 3.75s	remaining: 2.19s
137:	learn: 1.0439379	total: 3.78s	remaining: 2.16s
138:	learn: 1.0437309	total: 3.81s	remaining: 2.13s
139:	learn: 1.0436437	total: 3.83s	remaining: 2.11s
140:	learn: 1.0435742	total: 3.87s	remaining: 2.08s
141:	learn: 1.0433821	total: 3.89s	remaining: 2.06s
142:	learn: 1.0433016	total: 3.92s	remaining: 2.03s
143:	learn: 1.0431291	total: 3.96s	remaining: 2s
144:	learn: 1.0429478	total: 3.98s	remaining: 1.98s
145:	learn: 1.0426973	total: 4.01s	remaining: 1.95s
146:	learn: 1.0426638	total: 4.04s	remaining: 1.92s
147:	learn: 1.0425987	total: 4.06s	remaining: 1.89s
148:	learn: 1.0424562	total: 4.09s	remaining: 1.86s
149:	learn: 1.0424020	total: 4.12s	remaining: 1.84s
150:	learn: 1.0422816	total: 4.14s	remaining: 1.81s
151:	learn: 1.0421867	total: 4.17s	remaining: 1.78s
152:	learn: 1.0420918	total: 4.19s	remaining: 1.75s
153:	learn: 1.0

80:	learn: 1.0704799	total: 2.17s	remaining: 3.65s
81:	learn: 1.0701722	total: 2.2s	remaining: 3.62s
82:	learn: 1.0697798	total: 2.23s	remaining: 3.59s
83:	learn: 1.0695509	total: 2.25s	remaining: 3.56s
84:	learn: 1.0693178	total: 2.29s	remaining: 3.56s
85:	learn: 1.0690572	total: 2.33s	remaining: 3.55s
86:	learn: 1.0688393	total: 2.35s	remaining: 3.52s
87:	learn: 1.0687499	total: 2.38s	remaining: 3.49s
88:	learn: 1.0684708	total: 2.4s	remaining: 3.46s
89:	learn: 1.0682228	total: 2.43s	remaining: 3.43s
90:	learn: 1.0679734	total: 2.45s	remaining: 3.4s
91:	learn: 1.0678590	total: 2.48s	remaining: 3.37s
92:	learn: 1.0676553	total: 2.5s	remaining: 3.34s
93:	learn: 1.0674834	total: 2.53s	remaining: 3.31s
94:	learn: 1.0672930	total: 2.56s	remaining: 3.29s
95:	learn: 1.0671045	total: 2.59s	remaining: 3.26s
96:	learn: 1.0668851	total: 2.61s	remaining: 3.23s
97:	learn: 1.0667213	total: 2.64s	remaining: 3.2s
98:	learn: 1.0665694	total: 2.66s	remaining: 3.17s
99:	learn: 1.0663530	total: 2.68s	re

StochasticMultiModel(N=100, clf=<class 'catboost.core.CatBoostRegressor'>,
                     params={'cat_features': ['material_code', 'company_code',
                                              'region', 'month'],
                             'depth': 4, 'learning_rate': 0.05,
                             'n_estimators': 217, 'random_state': 4158})

In [169]:
dill.dump(final_ml_pipe, file = open("pipe_CATBOOST_1.cbm", "wb"))

In [170]:
%%writefile predict.py

import pathlib
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
import dill
import math
BASE_LOG = 4 # math.e

MODEL_FILE = pathlib.Path(__file__).parent.joinpath("pipe_CATBOOST_1.cbm")

AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]
FTS_COLS = ['material_code', 'company_code', 'region', 'month', 'vol_tm1', 'country_grp_mean_10', 
             'avg_2', 'avg_3', 'avg_12', 'median2', 'median3', 'median4', 'median12', 'max_2', 'max_3']
TARGET = "target"

# 
def get_features(df: pd.DataFrame, month: pd.Timestamp, N=1) -> pd.DataFrame:
    """Calculate features for `month`."""

    start_period = month - pd.offsets.MonthBegin(N)
    end_period = month - pd.offsets.MonthBegin(1)

    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    # формируем лаги за N месяцев
    features[[f"vol_tm{i}" for i in range(N, 0, -1)]] = df.loc[:, start_period:end_period].copy()
 
    # Добавление ГРУППОВЫХ скользящих средних
    gr = "country"
    period = 10
    df2 = df.copy()
    df2[df2.columns.to_list()] = \
                           df2.groupby(level=gr).transform(lambda x: x.mean())
    grp_manager_roll_mean = df2.rolling(period, axis=1, min_periods=1)
    features = \
        features.join( grp_manager_roll_mean.mean().iloc[:, -1].rename(gr+"_grp_mean_"+str(period)))

    #  MEAN
    for period in [2,3,12]: #range(2,13,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.mean().iloc[:, -1].rename("avg_"+str(period)))

    #  median
    for period in [2,3,4,12]: # 10,11,12]: #range(2,13,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.median().iloc[:, -1].rename("median"+str(period)))
                 
    #  MAX
    for period in range(2,4,1):
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join(rolling.max().iloc[:, -1].rename("max_"+str(period)))

    features["month"] = month.month

    return features.reset_index()


def predict(df: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:

    model = dill.load(open(MODEL_FILE, "rb"))

    group_ts = df.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)
    
    tmp = group_ts.clip(lower=1.0e-200).copy()
    for i in tmp.columns:
        tmp[i] = tmp[i].apply(lambda x: math.log(x+1, BASE_LOG))
    
    features = get_features(tmp, month)

    for c in features.columns:
        col_type = features[c].dtype
        if col_type == 'object': 
            features[c] = features[c].astype('category')    
    
    predictions = model.predict(features[FTS_COLS])
    predictions = BASE_LOG**predictions - 1
    
    predictions = pd.Series(predictions).clip(lower=0, upper=7000)

    preds_df = features[AGG_COLS].copy()
    preds_df["prediction"] = predictions
    return preds_df

Overwriting predict.py


In [171]:
import predict
import importlib
importlib.reload(predict)

<module 'predict' from 'C:\\Users\\cvirk\\Sibur2021\\predict.py'>

In [172]:
ts_preds_tst = predict.predict(data[data.month<"2020-07-01"], pd.Timestamp("2020-07-01"))
ts_preds_tst 

,material_code,company_code,country,region,manager_code,prediction
0,124,7278,Россия,Респ. Татарстан,17460,0.860632
1,133,0,Белоруссия,Минская обл.,10942,149.399785
2,133,0,Белоруссия,Могилевская обл.,10942,5.577041
3,133,0,Белоруссия,г. Минск,10942,1.952555
4,133,0,Казахстан,г. Нур-Султан,13301,16.005359
...,...,...,...,...,...,...
936,986,9943,Россия,Смоленская обл.,17460,32.653440
937,998,0,Россия,Ленинградская обл.,18079,1.894488
938,998,3380,Россия,Ленинградская обл.,14956,27.763080
939,998,5410,Россия,г. Санкт-Петербург,14956,64.361343


In [174]:
print("Ошибка на тестовом множестве:",
      f'{np.sqrt(mean_squared_log_error(group_ts.reset_index().iloc[:,-1], ts_preds_tst["prediction"])):.4f}')

Ошибка на тестовом множестве: 1.5355


In [175]:
# Windows
!tar.exe -a -c -f Multi7f5.zip pipe_CATBOOST_1.cbm requirements.txt predict.py

In [147]:
end_time = time()
time_taken = end_time - start_time # time_taken is in seconds
hours, rest = divmod(time_taken,3600)
minutes, seconds = divmod(rest, 60)
print('Время работы скрипта:: {} hours {} minutes {} seconds.'.format(hours, minutes, seconds))

Время работы скрипта:: 2.0 hours 19.0 minutes 18.570309162139893 seconds.


In [55]:
x_train, y_train = fulldata[FTS_COLS], \
y_fulldata.clip(lower=1.0e-200).apply(lambda x: math.log(x+1, BASE_LOG ))